# QUICK START:

In [1]:
!pip install mediapipe opencv-python


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [4]:
cap = cv2.VideoCapture(0)

# Rep counter variables
counter_L = 0
countdown_L = 100
# countdown_L = 36
stage_L = None

counter_R = 0
countdown_R = 100
# countdown_R = 36
stage_R = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # for bicep curls
            start_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            mid_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            end_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # for shoulder raises
            start_L = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            mid_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            end_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            start_R = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            mid_R = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            end_R = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            
            # Calculate angle
            angle_L = round(calculate_angle(start_L, mid_L, end_L), 2)
            angle_R = round(calculate_angle(start_R, mid_R, end_R), 2)
            
            # Visualize
            cv2.putText(image, str(angle_L), 
                        tuple(np.multiply(end_L, [640, 480]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(angle_R), 
                        tuple(np.multiply(end_R, [640, 480]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Rep counter logic
            # Left arm
            if angle_L < 50 :
                stage_L = "down"
                
            if angle_L > 90 and stage_L =='down':
                stage_L = "up"
                counter_L += 1
                countdown_L -= 1
                print("Reps (L)", counter_L)
                
                
            # Left arm
            if angle_R < 50 :
                stage_R = "down"
                
            if angle_R > 90 and stage_R =='down':
                stage_R = "up"
                counter_R += 1
                countdown_R -= 1
                print("Reps (R)", counter_R)

        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (150,85), (74,52,131), -1)
        cv2.rectangle(image, (180,0), (330,85), (74,52,131), -1)

    
        # Rep data - countdown
        cv2.putText(image, 'Reps (L)', (15, 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, str(countdown_L), (10, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.putText(image, 'Reps (R)', (195, 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, str(countdown_R), (190, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Rep data - counter
        # cv2.putText(image, 'Reps (L)', (15, 20), 
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        # cv2.putText(image, str(counter_L), (10, 75),
        #            cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # cv2.putText(image, 'Reps (R)', (195, 20), 
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        # cv2.putText(image, str(counter_R), (190, 75),
        #            cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(42,42,42), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(74,52,131), thickness=2, circle_radius=2)  
                                 )
        
        
        cv2.imshow('AI Rep Counter App', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        if cv2.waitKey(10) & 0xFF == ord('m'):
            #increase remaining reps by 3 when 'm' is pressed
            countdown_R += 3
            countdown_L += 3

    cap.release()
    cv2.destroyAllWindows()


Reps (L) 1
Reps (R) 1
Reps (L) 2
Reps (R) 2
Reps (L) 3
Reps (R) 3
Reps (L) 4
Reps (R) 4
Reps (L) 5
Reps (R) 5
Reps (L) 6
Reps (R) 6
Reps (L) 7
Reps (R) 7
Reps (L) 8
Reps (R) 8
Reps (L) 9
Reps (R) 9
Reps (L) 10
Reps (R) 10
Reps (L) 11
Reps (R) 11
Reps (L) 12
Reps (R) 12
Reps (L) 13
Reps (L) 14
Reps (R) 13
Reps (L) 15
Reps (R) 14
Reps (L) 16
Reps (R) 15
Reps (R) 16
Reps (L) 17
Reps (R) 17
Reps (L) 18
Reps (R) 18
Reps (L) 19
Reps (R) 19
Reps (L) 20
Reps (R) 20
Reps (L) 21
Reps (R) 21
Reps (L) 22
Reps (R) 22
Reps (R) 23
Reps (L) 23
Reps (L) 24
Reps (R) 24
Reps (L) 25
Reps (R) 25
Reps (L) 26
Reps (R) 26
Reps (L) 27
Reps (R) 27
Reps (L) 28
Reps (R) 28
Reps (L) 29
Reps (R) 29
Reps (L) 30
Reps (R) 30
Reps (L) 31
Reps (R) 31
Reps (L) 32
Reps (R) 32
Reps (L) 33
Reps (R) 33
Reps (L) 34
Reps (R) 34
Reps (L) 35
Reps (R) 35
Reps (L) 36
Reps (L) 37
Reps (L) 38
Reps (L) 39
Reps (L) 40
Reps (R) 36
Reps (R) 37
Reps (R) 38
Reps (R) 39
Reps (R) 40
Reps (R) 41
Reps (R) 42
Reps (R) 43
Reps (R) 44
Reps (R) 4

# 0. Install and Import Dependencies

In [9]:
!pip install mediapipe opencv-python

In [11]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [16]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('AI Rep Counter App', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [25]:
cap = cv2.VideoCapture(0)
# Setup mediapipe instance

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(69,69,69), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2)  
                                 )
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Determining Joints

In [18]:
cap = cv2.VideoCapture(0)
# Setup mediapipe instance

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(69,69,69), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2)  
                                 )
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.641379714012146
y: 0.0813635066151619
z: -0.487846314907074
visibility: 0.9999748468399048
, x: 0.6549774408340454
y: 0.049682021141052246
z: -0.4549216330051422
visibility: 0.9999505281448364
, x: 0.6647886037826538
y: 0.05392944812774658
z: -0.4549482464790344
visibility: 0.9999483823776245
, x: 0.6743061542510986
y: 0.05809438228607178
z: -0.45476406812667847
visibility: 0.9999358654022217
, x: 0.6272463798522949
y: 0.04175734519958496
z: -0.4599648118019104
visibility: 0.9999657869338989
, x: 0.6166039109230042
y: 0.04081404209136963
z: -0.45997709035873413
visibility: 0.9999698400497437
, x: 0.6057469844818115
y: 0.040816545486450195
z: -0.460143506526947
visibility: 0.9999644756317139
, x: 0.68427574634552
y: 0.07982128113508224
z: -0.24646666646003723
visibility: 0.9999290704727173
, x: 0.582390308380127
y: 0.06698233634233475
z: -0.27614980936050415
visibility: 0.9999762773513794
, x: 0.6548696160316467
y: 0.12398559600114822
z: -0.41173499822616577
visibility: 0.99996769

[x: 0.6429644227027893
y: 0.10295452922582626
z: -0.5232427716255188
visibility: 0.9999813437461853
, x: 0.6573442816734314
y: 0.06896008551120758
z: -0.49197012186050415
visibility: 0.9999648928642273
, x: 0.6677613854408264
y: 0.07057790458202362
z: -0.4921683669090271
visibility: 0.9999634623527527
, x: 0.6775813102722168
y: 0.07280934602022171
z: -0.4920772910118103
visibility: 0.9999548196792603
, x: 0.6284969449043274
y: 0.06584064662456512
z: -0.4922487139701843
visibility: 0.9999748468399048
, x: 0.6168465614318848
y: 0.06482920795679092
z: -0.49222859740257263
visibility: 0.9999775290489197
, x: 0.6052321195602417
y: 0.06465711444616318
z: -0.4923390746116638
visibility: 0.9999731183052063
, x: 0.6944273710250854
y: 0.09513931721448898
z: -0.26534515619277954
visibility: 0.9999513030052185
, x: 0.5869544744491577
y: 0.08656430244445801
z: -0.28134945034980774
visibility: 0.9999818801879883
, x: 0.6569974422454834
y: 0.14484351873397827
z: -0.4403308928012848
visibility: 0.9999

[x: 0.6424604654312134
y: 0.13750652968883514
z: -0.5732160806655884
visibility: 0.9999517798423767
, x: 0.6566976308822632
y: 0.10591011494398117
z: -0.5404589772224426
visibility: 0.9999306201934814
, x: 0.6653154492378235
y: 0.10783327370882034
z: -0.5405855774879456
visibility: 0.9999223351478577
, x: 0.675075352191925
y: 0.11066367477178574
z: -0.5408115386962891
visibility: 0.9999148845672607
, x: 0.6285404562950134
y: 0.10446856170892715
z: -0.5416083335876465
visibility: 0.9999475479125977
, x: 0.6172065138816833
y: 0.10517196357250214
z: -0.5415734052658081
visibility: 0.999946117401123
, x: 0.6060458421707153
y: 0.10689368844032288
z: -0.5417916178703308
visibility: 0.9999439716339111
, x: 0.6919527053833008
y: 0.13312073051929474
z: -0.3064572215080261
visibility: 0.999925971031189
, x: 0.5928548574447632
y: 0.1266249418258667
z: -0.30536097288131714
visibility: 0.9999656677246094
, x: 0.6567471027374268
y: 0.17410479485988617
z: -0.4867019057273865
visibility: 0.99996334314

[x: 0.6420350074768066
y: 0.1630750596523285
z: -0.4940832555294037
visibility: 0.9999234080314636
, x: 0.655661940574646
y: 0.13468393683433533
z: -0.46473413705825806
visibility: 0.9998797178268433
, x: 0.6631477475166321
y: 0.1354491263628006
z: -0.46463751792907715
visibility: 0.9998655915260315
, x: 0.6711359620094299
y: 0.13643857836723328
z: -0.46467870473861694
visibility: 0.9998552799224854
, x: 0.6284089684486389
y: 0.1337774246931076
z: -0.4732670187950134
visibility: 0.9999192357063293
, x: 0.6174545288085938
y: 0.13351617753505707
z: -0.47318291664123535
visibility: 0.9999170899391174
, x: 0.6076663732528687
y: 0.133502796292305
z: -0.4733182489871979
visibility: 0.9999178051948547
, x: 0.6833196878433228
y: 0.15517479181289673
z: -0.2574552297592163
visibility: 0.9998491406440735
, x: 0.5950143933296204
y: 0.15107662975788116
z: -0.29371708631515503
visibility: 0.9999560117721558
, x: 0.6559954881668091
y: 0.19762788712978363
z: -0.4165833592414856
visibility: 0.999939560

[x: 0.6241483092308044
y: 0.1741447001695633
z: -0.46282505989074707
visibility: 0.9999378323554993
, x: 0.6363691687583923
y: 0.14590002596378326
z: -0.42915263772010803
visibility: 0.9999028444290161
, x: 0.6440891027450562
y: 0.14719761908054352
z: -0.42924484610557556
visibility: 0.9998921155929565
, x: 0.6508151292800903
y: 0.14890159666538239
z: -0.42931729555130005
visibility: 0.9998840093612671
, x: 0.6125866174697876
y: 0.14505550265312195
z: -0.4349382519721985
visibility: 0.9999343156814575
, x: 0.6045693159103394
y: 0.1450982689857483
z: -0.43490996956825256
visibility: 0.9999322891235352
, x: 0.5968605875968933
y: 0.1455834060907364
z: -0.43503302335739136
visibility: 0.9999328255653381
, x: 0.6589982509613037
y: 0.1693759709596634
z: -0.20733873546123505
visibility: 0.9998793005943298
, x: 0.5842917561531067
y: 0.16470129787921906
z: -0.233943909406662
visibility: 0.9999637603759766
, x: 0.6385398507118225
y: 0.20687136054039001
z: -0.381399542093277
visibility: 0.9999486

...

[x: 0.5990498065948486
y: 0.166044220328331
z: -0.40945887565612793
visibility: 0.9999626874923706
, x: 0.6134712100028992
y: 0.1398022174835205
z: -0.3823552429676056
visibility: 0.9999406337738037
, x: 0.6212111711502075
y: 0.14088374376296997
z: -0.3825058043003082
visibility: 0.9999431371688843
, x: 0.6292027235031128
y: 0.14262054860591888
z: -0.3825267553329468
visibility: 0.9999383091926575
, x: 0.5890365839004517
y: 0.13978688418865204
z: -0.37915730476379395
visibility: 0.9999609589576721
, x: 0.5804787874221802
y: 0.14075691998004913
z: -0.37917783856391907
visibility: 0.999962568283081
, x: 0.5725392699241638
y: 0.142507404088974
z: -0.3792819380760193
visibility: 0.9999595284461975
, x: 0.6438885927200317
y: 0.1629595011472702
z: -0.18821261823177338
visibility: 0.9999427199363708
, x: 0.5633260011672974
y: 0.16375349462032318
z: -0.1791522204875946
visibility: 0.9999692440032959
, x: 0.6152475476264954
y: 0.198802188038826
z: -0.3375076949596405
visibility: 0.9999617338180

[x: 0.6240506768226624
y: 0.1444597691297531
z: -0.4169082045555115
visibility: 0.9999643564224243
, x: 0.6388706564903259
y: 0.11953434348106384
z: -0.39077070355415344
visibility: 0.9999337196350098
, x: 0.6463640928268433
y: 0.12117566168308258
z: -0.3908591866493225
visibility: 0.9999442100524902
, x: 0.6542741060256958
y: 0.12335129082202911
z: -0.39085885882377625
visibility: 0.999933123588562
, x: 0.617162823677063
y: 0.11736489087343216
z: -0.3901996910572052
visibility: 0.9999549984931946
, x: 0.6097071766853333
y: 0.11727973073720932
z: -0.39026308059692383
visibility: 0.9999611377716064
, x: 0.6020197868347168
y: 0.11790058016777039
z: -0.3904341161251068
visibility: 0.9999532699584961
, x: 0.6672543883323669
y: 0.14522705972194672
z: -0.2115146815776825
visibility: 0.9999374151229858
, x: 0.5911271572113037
y: 0.13898898661136627
z: -0.21039478480815887
visibility: 0.9999733567237854
, x: 0.6369208693504333
y: 0.17945535480976105
z: -0.3510664403438568
visibility: 0.9999586

[x: 0.6187133193016052
y: 0.11275100708007812
z: -0.568034291267395
visibility: 0.999911904335022
, x: 0.6364510655403137
y: 0.07872389256954193
z: -0.5375667810440063
visibility: 0.9998712539672852
, x: 0.6460669636726379
y: 0.08047180622816086
z: -0.5376060605049133
visibility: 0.9998709559440613
, x: 0.6555661559104919
y: 0.08324825763702393
z: -0.537653923034668
visibility: 0.9998542070388794
, x: 0.6101315021514893
y: 0.07705556601285934
z: -0.5279718637466431
visibility: 0.9999150037765503
, x: 0.6021040081977844
y: 0.07721330970525742
z: -0.5279430747032166
visibility: 0.9999188780784607
, x: 0.5944047570228577
y: 0.07815210521221161
z: -0.5280873775482178
visibility: 0.9999128580093384
, x: 0.6735371947288513
y: 0.1016273945569992
z: -0.3287736177444458
visibility: 0.9998870491981506
, x: 0.5855926871299744
y: 0.0962771400809288
z: -0.28077876567840576
visibility: 0.9999523758888245
, x: 0.6354764103889465
y: 0.15204352140426636
z: -0.49199533462524414
visibility: 0.99993145465

[x: 0.6284660696983337
y: 0.04940378665924072
z: -0.5488959550857544
visibility: 0.9995414018630981
, x: 0.6445686221122742
y: 0.019726164638996124
z: -0.5230521559715271
visibility: 0.9991894364356995
, x: 0.6552309989929199
y: 0.024261794984340668
z: -0.5231912136077881
visibility: 0.9991550445556641
, x: 0.6653832197189331
y: 0.029185350984334946
z: -0.5232707262039185
visibility: 0.9991260766983032
, x: 0.61546391248703
y: 0.016445942223072052
z: -0.518873929977417
visibility: 0.9995034337043762
, x: 0.6062361001968384
y: 0.018698502331972122
z: -0.519041895866394
visibility: 0.9994834065437317
, x: 0.5973490476608276
y: 0.02135581709444523
z: -0.5193039178848267
visibility: 0.9995196461677551
, x: 0.6778239607810974
y: 0.052022505551576614
z: -0.34226375818252563
visibility: 0.9991108179092407
, x: 0.5779480338096619
y: 0.04592765495181084
z: -0.3169972598552704
visibility: 0.9996753931045532
, x: 0.6454171538352966
y: 0.09210509806871414
z: -0.47960272431373596
visibility: 0.9996

[x: 0.5920520424842834
y: -0.025529181584715843
z: -0.6069787740707397
visibility: 0.9945138096809387
, x: 0.6051250696182251
y: -0.06380809843540192
z: -0.5764423608779907
visibility: 0.9897466897964478
, x: 0.6166731715202332
y: -0.06616172939538956
z: -0.5765188932418823
visibility: 0.9878291487693787
, x: 0.6283696889877319
y: -0.06841195374727249
z: -0.5766124725341797
visibility: 0.9888764023780823
, x: 0.5675939321517944
y: -0.06043523550033569
z: -0.5824841260910034
visibility: 0.9931812286376953
, x: 0.5544298887252808
y: -0.05707914009690285
z: -0.5826689600944519
visibility: 0.99249267578125
, x: 0.5425512790679932
y: -0.05358056351542473
z: -0.5832560658454895
visibility: 0.9936569929122925
, x: 0.6421471834182739
y: -0.05059177428483963
z: -0.36925065517425537
visibility: 0.9871335029602051
, x: 0.5288234949111938
y: -0.027501177042722702
z: -0.40321677923202515
visibility: 0.9946810603141785
, x: 0.620409369468689
y: 0.019813988357782364
z: -0.5325409173965454
visibility:

In [32]:
for lndmark in mp_pose.PoseLandmark:
    print(lndmark)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [33]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.3888421654701233
y: 0.6245141625404358
z: -0.565764307975769
visibility: 0.9989931583404541

In [36]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.3316383957862854
y: 0.9838322997093201
z: -0.6861981153488159
visibility: 0.4678402245044708

In [37]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.14278361201286316
y: 1.3525813817977905
z: -0.6563081741333008
visibility: 0.08514130115509033

In [35]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value

11

# 3. Calculate and Render Angles

In [19]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

print(calculate_angle(shoulder_L, elbow_L, wrist_L))

177.61062248476463


In [44]:
shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [20]:
shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

print(calculate_angle(shoulder_L, elbow_L, wrist_L))

177.61062248476463


In [21]:
cap = cv2.VideoCapture(0)
# Setup mediapipe instance

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # for bicep curls
            shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # for shoulder raises
            shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            
            # Calculate angle
            angle = round(calculate_angle(shoulder_L, elbow_L, wrist_L), 2)
            print("WORKING")
            print(angle)
            
            # Visualize
            cv2.putText(image, str(angle), 
                        tuple(np.multiply(elbow_L, [640, 480]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(69,69,69), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2)  
                                 )
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

WORKING
17.07
WORKING
16.39
WORKING
16.0
WORKING
14.94
WORKING
14.48
WORKING
13.98
WORKING
13.35
WORKING
13.39
WORKING
13.09
WORKING
12.6
WORKING
12.64
WORKING
12.04
WORKING
11.69
WORKING
11.61
WORKING
12.09
WORKING
11.79
WORKING
12.04
WORKING
12.02
WORKING
12.24
WORKING
12.16
WORKING
12.37
WORKING
12.18
WORKING
12.02
WORKING
12.76
WORKING
13.64
WORKING
15.22
WORKING
15.32
WORKING
16.27
WORKING
16.77
WORKING
17.59
WORKING
17.19
WORKING
14.81
WORKING
16.51
WORKING
16.81
WORKING
17.96
WORKING
16.08
WORKING
16.57
WORKING
16.41
WORKING
21.93
WORKING
24.64
WORKING
26.42
WORKING
26.59
WORKING
24.39
WORKING
26.64
WORKING
26.86
WORKING
20.62
WORKING
22.09
WORKING
16.69
WORKING
17.79
WORKING
19.26
WORKING
23.97
WORKING
25.72
WORKING
32.46
WORKING
35.38
WORKING
41.9
WORKING
45.39
WORKING
43.71
WORKING
44.52
WORKING
44.85
WORKING
38.32
WORKING
34.56
WORKING
34.76
WORKING
36.65
WORKING
39.08
WORKING
42.82
WORKING
45.37
WORKING
45.59
WORKING
44.1
WORKING
42.43
WORKING
30.99
WORKING
25.67
WORKING
22

# 4. Rep Counter Logic and Render

In [23]:
cap = cv2.VideoCapture(0)

# Rep counter variables
counter_L = 0
countdown_L = 100
stage_L = None

counter_R = 0
countdown_R = 100
stage_R = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # for bicep curls
            start_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            mid_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            end_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # for shoulder raises
            start_L = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            mid_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            end_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            start_R = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            mid_R = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            end_R = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            
            # Calculate angle
            angle_L = round(calculate_angle(start_L, mid_L, end_L), 5)
            angle_R = round(calculate_angle(start_R, mid_R, end_R), 5)
            
            # Visualize
            cv2.putText(image, str(angle_L), 
                        tuple(np.multiply(end_L, [640, 480]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, str(angle_R), 
                        tuple(np.multiply(end_R, [640, 480]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Rep counter logic
            # Left arm
            if angle_L < 50 :
                stage_L = "down"
                
            if angle_L > 90 and stage_L =='down':
                stage_L = "up"
                counter_L += 1
                countdown_L -= 1
                print(counter_L)
                
                
            # Left arm
            if angle_R < 50 :
                stage_R = "down"
                
            if angle_R > 90 and stage_R =='down':
                stage_R = "up"
                counter_R += 1
                countdown_R -= 1
                print(counter_R)

        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (150,85), (245,117,16), -1)
        cv2.rectangle(image, (180,0), (330,85), (245,117,169), -1)

    
        # Rep data
        cv2.putText(image, 'Reps (L)', (15, 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, str(countdown_L), (10, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.putText(image, 'Reps (R)', (195, 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, str(countdown_R), (190, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(69,69,69), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2)  
                                 )
        
        
        cv2.imshow('AI Rep Counter App', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
1
2
7
3
8
4
9
5
10
6
11
7
12
8
13
9
14
10
15
11
16
12
17
13
18
14
19
15
20
16
21
17
22
23
18
19
20
24
21
22
25
23
26
24
27
25
28
26
29
27
30
28
31
29
32
33
30
31
32
33
34
35
34
35
36
36
37
37
38
38
39
40
39
41
40
41
42
43
42
44
43
45
44
45
46
47
48
49
